In [3]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
from sklearn.model_selection import train_test_split

# === 1. Wczytaj dane z Excela ===
df = pd.read_excel("SampleInfo_fixed_08_07_23.xlsx", sheet_name="Sheet1")

# Wyodrębnij kolumny
X_raw = df[["lib.size"]].to_numpy()
Y_raw = df["originalScoreBinary"].apply(lambda x: 1 if x >= 0.5 else 0).to_numpy()

# Normalizacja do zakresu [0, 1]
X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
Y = Y_raw

# Split danych
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# === 2. PennyLane model ===
n_qubits = 1
dev = qml.device("default.qubit", wires=n_qubits)

def state_preparation(x):
    qml.RX(x[0] * np.pi, wires=0)

def variational_circuit(weights):
    qml.RY(weights[0], wires=0)
    qml.RZ(weights[1], wires=0)

@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)
    variational_circuit(weights)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, x):
    return circuit(weights, x)

def square_loss(labels, predictions):
    return np.mean((labels - predictions) ** 2)

def accuracy(labels, predictions):
    predictions = np.sign(predictions)
    return np.mean(labels == (predictions > 0))

def train(X, Y, epochs=25, lr=0.1):
    weights = np.random.randn(2, requires_grad=True)
    opt = qml.GradientDescentOptimizer(stepsize=lr)

    for i in range(epochs):
        def cost(w):
            preds = np.array([variational_classifier(w, x) for x in X])
            return square_loss(Y, preds)


        weights = opt.step(cost, weights)

        if i % 5 == 0:
            preds = [variational_classifier(weights, x) for x in X]
            acc = accuracy(Y, preds)
            print(f"Epoch {i}: loss = {cost(weights):.4f}, accuracy = {acc:.4f}")

    return weights

# === 3. Uruchom trening ===
trained_weights = train(X_train, Y_train)

# === 4. Test na danych testowych ===
predictions = [variational_classifier(trained_weights, x) for x in X_test]
test_accuracy = accuracy(Y_test, predictions)
print(f"\nTest accuracy: {test_accuracy:.4f}")


Epoch 0: loss = 0.2070, accuracy = 0.7702
Epoch 5: loss = 0.2001, accuracy = 0.7702
Epoch 10: loss = 0.1974, accuracy = 0.7702
Epoch 15: loss = 0.1961, accuracy = 0.7702
Epoch 20: loss = 0.1955, accuracy = 0.7702

Test accuracy: 0.7728
